In [ ]:
import os
import getpass
from dotenv import load_dotenv
from langchain import hub
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from typing import List
from langchain_core.documents import Document

In [ ]:
load_dotenv()

class Config:
    GROQ_API_KEY = os.getenv("GROQ_API_KEY") or getpass.getpass("Groq API Key: ")
    FAST_API_URL = os.getenv("API_URL") or getpass.getpass("FastAPI URL: ")
    INDEX_NAME = "invoice-analysis"
    EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"
    LLM_MODEL = "llama3-70b-8192"
    LLM_TEMPERATURE = 0.3
    VECTOR_STORE_DIR = "E:\\RAGbot\\vectorDB"
    DB_NAME = "invoice_analysis_report"

config = Config()


class VectorStore:
    def __init__(self, db_path: str = config.VECTOR_STORE_DIR) -> None:
        self.embeddings = HuggingFaceEmbeddings(model_name=config.EMBEDDING_MODEL)
        self.vector_store = Chroma(
            collection_name=config.DB_NAME,
            embedding_function=self.embeddings,
            persist_directory='E:\\RAGbot\\vectorDB'
        )
        
    def similarity_search(self, query: str, k: int = 4) -> List[Document]:
        """Search for similar documents"""
        return self.vector_store.similarity_search(query, k=k)
    
    def as_retriver(self):
        return self.vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5})
        

In [ ]:
graph_builder = StateGraph(MessagesState)
llm = init_chat_model("llama3-8b-8192", model_provider="groq")
prompt = hub.pull("rlm/rag-prompt")
config = Config()
vector_store = VectorStore()

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [30]:
# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [31]:
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [32]:
input_message = "Hello, is Reetu is a Customer name"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hello, is Reetu is a Customer name
================================== Ai Message ==================================
Tool Calls:
  retrieve (6ddtsdvn4)
 Call ID: 6ddtsdvn4
  Args:
    query: Is Reetu a Customer name
================================= Tool Message =================================
Name: retrieve

Source: {'status': 'accept', 'employee_name': 'Reetu', 'date': '10 Sep 2024', 'invoice_id': '397373', 'reason': 'The reimbursement request is in compliance with the policy guidelines. The total amount of ₹233 is within the approved limits for travel expenses, and the invoice is supported by the required documentation.'}
Content: Invoice Content: Original Tax Invoice Driver Trip Invoice Sunil Service Tax Category: Renting of Cab Cab 2F A1341 Invoice ID 397373 Invoice Date 10 Sep 2024 Customer Name Reetu Mobile Number 8901233212 Pickup Address Rajaji Nagar Descri`ption A mount (₹) Ride F ee ₹ 187.46 T

In [55]:
metadata_filter ={}

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
message = "Hello, is Reetu is a Customer name"
input_messages = [HumanMessage(content=message)]

result = graph.invoke({
            "messages": input_messages,
            
        })


In [46]:
result

{'messages': [HumanMessage(content='Hello, is Reetu is a Customer name', additional_kwargs={}, response_metadata={}, id='223086bd-0201-4efc-ac4e-2f8fdb29b211'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 's1hmncxzs', 'function': {'arguments': '{"query":"Is Reetu a Customer name"}', 'name': 'retrieve'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 884, 'total_tokens': 954, 'completion_time': 0.125933391, 'prompt_time': 0.101201918, 'queue_time': 0.265366175, 'total_time': 0.227135309}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8dc6ecaf8e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d1e316f7-0175-4268-8f6b-ef5d352d00c7-0', tool_calls=[{'name': 'retrieve', 'args': {'query': 'Is Reetu a Customer name'}, 'id': 's1hmncxzs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 884, 'output_tokens': 70, 'total_tokens': 954}),
  ToolMessage(content="Source: {'invoice_id': '397373', 'em

In [47]:
ai_messages = [msg for msg in result['messages'] if msg.type == 'ai']
ai_messages

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 's1hmncxzs', 'function': {'arguments': '{"query":"Is Reetu a Customer name"}', 'name': 'retrieve'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 884, 'total_tokens': 954, 'completion_time': 0.125933391, 'prompt_time': 0.101201918, 'queue_time': 0.265366175, 'total_time': 0.227135309}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8dc6ecaf8e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d1e316f7-0175-4268-8f6b-ef5d352d00c7-0', tool_calls=[{'name': 'retrieve', 'args': {'query': 'Is Reetu a Customer name'}, 'id': 's1hmncxzs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 884, 'output_tokens': 70, 'total_tokens': 954}),
 AIMessage(content='Yes, Reetu is a customer name.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 548, 'total_tokens': 558, 'completion_time': 0.018234502, 'prompt_tim

In [57]:
from typing import Optional
from pydantic import BaseModel
import json
ai_messages = [msg for msg in result['messages'] if msg.type == 'ai']
final_response = ai_messages[-1].content

class ChatResponse(BaseModel):
    response: str
    status: str
    details: Optional[str] = None

if not ai_messages:
    res=ChatResponse(
            status="error",
            response="No response generated by the chatbot",
            details=str(result)
        )
else:
    res= ChatResponse(
            status="success",
            response=final_response,
            metadata=metadata_filter
        )

res.model_dump_json()

'{"response":"Yes, Reetu is a customer name.","status":"success","details":null}'

In [58]:
print(res)

response='Yes, Reetu is a customer name.' status='success' details=None
